# Lab 5. Natural Language Processing. Unsupervised Learning

In [0]:
# Some IPython magic
# Put these at the top of every notebook, here nbagg is used for interactive plots
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

## Natural Language Processing.
NLP refers to processing text data. This could refer to a wide range of tasks, from very simple ones, like searching for a pattern, to very complex ones, like text summarization, or automated translation.

### Feature Extraction
In order to apply Machine Learning algorithms on text data, we need to figure out a way to represent the text as a set of numeric attributes.

#### Bag of Words
The simplest way to represent a text document as a vector of numbers is to count the words, and output a frequency count. Let's say we have a list of all english words, like the following:

In [38]:
# creates a wordlist, with all words, from "a" to "zygote"
import urllib.request as request
words = request.urlopen("https://svnweb.freebsd.org/csrg/share/dict/words?view=co")
wordlist = []
for w in words:
    wordlist.append(str(w.decode().strip()))
print(', '.join(wordlist[:4]) + " ... " + ', '.join(wordlist[-4:]))

a, AAA, AAAS, aardvark ... zucchini, Zulu, Zurich, zygote


In [0]:
print("Now we can convert any text to a vector of size " \
      + str(len(wordlist)))

For example, the text "In this lab we study Natural Language Processing and Unsupervised Learning" can be represented as a vector with almost all values equal to 0, and values of 1 in the position of the words "in", "this", etc.

This ___feature vector___ can be extracted directly from the dataset. If we have a large collection of text, we can assume that other documents will use the same vocabulary. So if we build a model for news articles, most likely those articles will not use every single word in the english language. So during the training phase of our machine learning modeling, we can use the train set to create our feature vector. We select only the words that appear in the train set. If new words appear during the test phase, we will discard them. This is a good thing because during training, we did not learn anything about those words. We cannot use unseen words to perform classification.

Let's start working with a dataset.

In [0]:
from sklearn.datasets import fetch_20newsgroups

# We select only 3 categories for now, feel free to change the categories
categories = [
    'rec.sport.baseball',
    'comp.graphics',
    'sci.space',
]
dataset = fetch_20newsgroups(subset='all', categories=categories, 
                              shuffle=True, random_state=42)

#dataset = fetch_20newsgroups(subset='all', shuffle=True, random_state=42) #if you want all caterogies


In [0]:
# here are the attributes of the object retrieved by fetch_20newsgroups
dir(dataset)

In [0]:
len(dataset.data)

In [0]:
X = dataset.data
y = dataset.target

In [0]:
print(dataset.data[0])

In [0]:
dataset.target_names

One way to turn a text document into a feature vector is to use a frequency count of
each word in the document.  We build a large dictionary of words, and for each document
we return a vector with as many features as there are words, and for each word, we return
the  number  of  times  that  word  appears  in  the  document  (this  is  technically  called
**term frequency** , or tf for short).  Sklearn has a **[CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)**  that does just that.

In [41]:
#TO DO: Transfrom the dataset into numerical feature vectors
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(dataset.data)
vocabulary = vectorizer.get_feature_names()
print(vocabulary[:10])
print(vocabulary[-10:])

['00', '000', '0000', '00000', '000000', '000005102000', '000021', '000050', '000062david42', '0000ahc']
['zwarte', 'zwork', 'zyda', 'zyeh', 'zyxel', 'zz', 'zzzzzz', 'zzzzzzt', 'ªl', 'ñaustin']


We can see there are many examples with digits or special characters ('ñaustin'). We could try to improve the vectorizer by removing some characters, but we will do this with TfIdf below.

One problem with this representation is the high frequency of common words like ”the” or ”to” or ”and”.  Those words appear in almost all documents, so they don’t offer much information.
A better way to extract features from text is to use both the **term frequency** metric and the **inverse document frequency** metric . Sklearn has a **[TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)** that does just that.

In [0]:
#TO DO: Transfrom the dataset into tf-idf feature vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset.data)
vocabulary = vectorizer.get_feature_names()
print(vocabulary[:10])
print(vocabulary[-10:])

['00', '000', '0000', '00000', '000000', '000005102000', '000021', '000050', '000062david42', '0000ahc']
['zwarte', 'zwork', 'zyda', 'zyeh', 'zyxel', 'zz', 'zzzzzz', 'zzzzzzt', 'ªl', 'ñaustin']


Let's improve the embedding by tweaking the TfidfVectorizer.

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='ascii', stop_words='english', token_pattern=r'(?u)\b[A-Za-z]+\b')
X = vectorizer.fit_transform(dataset.data)
vocabulary = vectorizer.get_feature_names()
print(vocabulary[:10])
print(vocabulary[-10:])

['aa', 'aaa', 'aaaa', 'aaaaarrrrgh', 'aaai', 'aaauuugggghhhhh', 'aachen', 'aacs', 'aad', 'aalborg']
['zwakke', 'zware', 'zwarte', 'zwork', 'zyda', 'zyeh', 'zyxel', 'zz', 'zzzzzz', 'zzzzzzt']


And now we have words with only alphabetical lowercase characters.

Now you will need to use the vectorized dataset to perfom clustering.

You will need to  use the following algorithms : [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html), [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.k_means.html#sklearn.cluster.k_means), [AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering).

For each algorithm try to find the parameters that produce clusters as similar as possible to the real distribution of the data.

Use different metrics to evaluate the algorithms : [Rand Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html),  [Silhouette Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html), [Homogeneity Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html), [Completness Score.](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.completeness_score.html#sklearn.metrics.completeness_score)






In [0]:
# TODO : Use the following algorithms to perform clustering on the dataset  
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score, homogeneity_score, completeness_score


In [44]:
# DBSCAN out of the box:
dbscan = DBSCAN()
dbscan.fit(X)
unique, counts = np.unique(dbscan.labels_, return_counts=True)
print("clusters:", dict(zip(unique, counts)))


clusters: {-1: 2941, 0: 6, 1: 7}


Most points are marked as outliers. Clearly we should increase the radious. So let's try different values and store the clusterings.

In [47]:
# we make a list of clusterings and store each clustering as a dictionary 
# with relevant information.

clusterings = []

epsilon = np.arange(0.7, 2, 0.1)
for e in epsilon:
  cluster = {}
  dbscan = DBSCAN(eps=e)
  dbscan.fit(X)
  cluster["algorithm"] = "DBSCAN"
  cluster["labels"] = dbscan.labels_
  cluster["parameters"] = {'eps': e}
  clusterings.append(cluster)
  
  unique, counts = np.unique(dbscan.labels_, return_counts=True)
  print("eps:",e,"clusters:", dict(zip(unique, counts)))

eps: 0.7 clusters: {-1: 2924, 0: 5, 1: 6, 2: 6, 3: 7, 4: 6}
eps: 0.7999999999999999 clusters: {-1: 2884, 0: 6, 1: 5, 2: 6, 3: 6, 4: 6, 5: 6, 6: 7, 7: 7, 8: 6, 9: 5, 10: 5, 11: 5}
eps: 0.8999999999999999 clusters: {-1: 2772, 0: 8, 1: 9, 2: 7, 3: 8, 4: 6, 5: 7, 6: 7, 7: 11, 8: 8, 9: 6, 10: 6, 11: 6, 12: 7, 13: 8, 14: 6, 15: 5, 16: 5, 17: 9, 18: 5, 19: 5, 20: 5, 21: 6, 22: 5, 23: 6, 24: 6, 25: 5, 26: 5, 27: 5}
eps: 0.9999999999999999 clusters: {-1: 2502, 0: 5, 1: 8, 2: 9, 3: 12, 4: 8, 5: 7, 6: 7, 7: 17, 8: 7, 9: 21, 10: 8, 11: 7, 12: 8, 13: 7, 14: 21, 15: 7, 16: 8, 17: 8, 18: 14, 19: 7, 20: 6, 21: 7, 22: 8, 23: 6, 24: 7, 25: 9, 26: 6, 27: 5, 28: 6, 29: 5, 30: 5, 31: 5, 32: 7, 33: 21, 34: 6, 35: 10, 36: 6, 37: 5, 38: 9, 39: 7, 40: 5, 41: 5, 42: 7, 43: 7, 44: 11, 45: 7, 46: 10, 47: 5, 48: 7, 49: 6, 50: 5, 51: 5, 52: 5, 53: 5, 54: 5, 55: 5, 56: 5, 57: 5}
eps: 1.0999999999999999 clusters: {-1: 1878, 0: 333, 1: 17, 2: 17, 3: 13, 4: 8, 5: 8, 6: 17, 7: 23, 8: 40, 9: 63, 10: 5, 11: 14, 12: 102, 1

We still don't obtain good results. We obtain either a lot of outliers, either many small clusters, or one big cluster.

In [0]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
unique, counts = np.unique(kmeans.labels_, return_counts=True)
print("clusters:", dict(zip(unique, counts)))

clusters: {0: 539, 1: 719, 2: 1696}


KMeans always produces the number of clusters we set it to produce. Let's vary the number of clusters and print the Silhouette Score. We can see from the documentation that this is the only metric of the mentioned ones that takes only the clustered data as input, and attempts to measure the fitness of the clustering. The other metrics compare a clustering with a ground truth. We will use them to test whether the clustering coincides with the topics of the documents.

In [0]:
scores = []
for n in range(2,10):
  kmeans = KMeans(n_clusters=n)
  kmeans.fit(X)
  s=silhouette_score(X,kmeans.labels_)
  agg_scores.append((n,s))
  print(n,':',s)
  
  cluster["algorithm"] = "KMeans"
  cluster["labels"] = kmeans.labels_
  cluster["parameters"] = {'n_clusters': n}

2 : 0.004192722109319261


All kmeans clusterings produce near zero silhouette scores. This means overlapping clusters.

In [55]:
# Agglomerative clustering takes too much to run mutiple times.
agg = AgglomerativeClustering(n_clusters=3)
agg.fit(X.toarray())
s=silhouette_score(X,agg.labels_)
print(s)

cluster["algorithm"] = "Agglomerative"
cluster["labels"] = agg.labels_
cluster["parameters"] = {'n_clusters': 3}

0.005265867921220907


Let's compare K-Means and Agglomerative Clustering with 3 clusters with the original classes.

In [56]:
relevant_clusterings = [c for c in clusterings 
                        if c['algorithm'] == 'KMeans' and 
                        c['parameters']['n_clusters'] == 3 or
                        c['algorithm'] == 'Agglomerative' and
                        c['parameters']['n_clusters'] == 3]

# for metric in [adjusted_rand_score, homogeneity_score, completeness_score]:
for c in relevant_clusterings:
  print(c['algorithm'],':')
  print('rand score :', adjusted_rand_score(y, c['labels']))
  print('homogenity :', homogeneity_score(y, c['labels']))
  print('completeness :', adjusted_rand_score(y, c['labels']))

Agglomerative :
rand score : 0.5659118196442221
homogenity : 0.5337938117724491
completeness : 0.5659118196442221


As you can see, high dimensional sparse vectors do not produce the best clusters.
Now, try to improve your results by using [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the dimensions of your feature vectors before applying the clustering algorithms. 





In [0]:
from sklearn.decomposition import PCA

Use t-SNE to produce a low-dimensional embedding of the dataset (and plot it).

In [0]:
from sklearn.manifold import TSNE

As an extra exercise, try to implement kernel KMeans. Look at the KMeans course. Slide 70

In [0]:
from sklearn.datasets import make_circles

X, _ = make_circles(n_samples=100, noise=0.1, factor=0)

In [0]:
from sklearn.metrics.pairwise import rbf_kernel


k = 2
# TODO : assign points to random clusters
y = 
dist = np.zeros((X.shape[0], k))

        
# TODO
max_iter = 10      
for _ in range(max_iter):      
  for j in range(k):
    # TODO : get the points that are in cluster j
    X_j = 
    
    # TODO : compute the first term
          
    first_term = 
    
    # TODO : compute the second term
    
    second_term = 
      
    dist[:, j] = first_term + second_term
        
  # TODO : change the clusters
  y = np.argmin(....)

   

In [0]:
plt.scatter(X[:,0], X[:,1], c=y)